Example workflow using pycaret

In [10]:
import pandas as pd
from .pycaret_model.weather_scrape import *
from electricity_scrape import *

ModuleNotFoundError: No module named 'weather_scrape'

In [ ]:
#aeso tables that can currently be scraped by function
dl_tables = ['DdsChargeSummary', 'HistoricalPoolPrice', 'ASPStandbyOperatingReservePrice',
             'ASPActivation', 'ASPActiveOperatingReservePrice', 'ActualForecastWMRQH', 
             'DailyAveragePoolPrice', 'HistoricalSystemMarginalPrice', 'RegulatingReserves']

In [ ]:
#input parameters
#aeso inputs
startdate = '2021-03-01'
enddate = '2021-03-30'
dateformat = '%Y-%m-%d'
contenttype = 'html'
#additional weather inputs city and timeframe 1 = hourly, 2 = daily
city = 'yyc'

In [ ]:
#scrape all available tables for defined time range
for t in dl_tables:
    #dataframe name is prefixed with "aeso_"
    df_name = 'aeso_'+t
    exec(df_name +" = aeso_download_range('" + t + "', contenttype, startdate, enddate, dateformat)")

In [ ]:
#scrape hourly weather data for defined timeframe
weather_hourly = weather_dl_range(startdate, enddate, dateformat, city, 1, clean=True)
#scrape daily weather data for defined timeframe
weather_daily = weather_dl_range(startdate, enddate, dateformat, city, 2, clean=True)

In [ ]:
#check the downloaded aeso table
for t in dl_tables:
    #dataframe name is prefixed with "aeso_"
    df_name = 'aeso_'+t
    print('Dataframe Name: '+df_name)
    exec("print(" + df_name + ".head())")

In [ ]:
weather_hourly.head()

In [11]:

from pycaret.datasets import get_data
dataset = get_data('diamond')

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


In [12]:
#check the shape of data
dataset.shape

(6000, 8)

In [13]:

data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (5400, 8)
Unseen Data For Predictions: (600, 8)


In [14]:
from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'Price', session_id=123, normalize = True)

,Description,Value
0,session_id,123
1,Target,Price
2,Original Data,"(5400, 8)"
3,Missing Values,False
4,Numeric Features,1
5,Categorical Features,6
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(3779, 28)"


In [15]:
best = compare_models(exclude = ['ransac'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,762.0118,2763999.1585,1612.2410,0.9729,0.0817,0.0607,0.1450
rf,Random Forest Regressor,760.1178,2926249.0594,1661.9234,0.9715,0.0817,0.0597,0.1310
lightgbm,Light Gradient Boosting Machine,752.8888,3056883.9503,1688.2518,0.9711,0.0773,0.0567,0.0140
gbr,Gradient Boosting Regressor,920.5793,3764751.6131,1901.5496,0.9633,0.1024,0.0770,0.0340
dt,Decision Tree Regressor,1002.1972,5302467.7551,2227.7463,0.9476,0.1082,0.0774,0.0050
knn,K Neighbors Regressor,1768.8920,10880444.0500,3273.0704,0.8926,0.2217,0.1641,0.0140
ridge,Ridge Regression,2415.2563,14120120.5000,3726.2067,0.8621,0.6687,0.2879,0.0050
lasso,Lasso Regression,2412.4036,14246721.3000,3744.2323,0.8608,0.6741,0.2866,0.1430
llar,Lasso Least Angle Regression,2355.6149,14272020.3989,3745.3094,0.8607,0.6391,0.2728,0.0040
lr,Linear Regression,2413.1627,14256607.9000,3745.8694,0.8606,0.6683,0.2868,0.2690


In [16]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [17]:
ada = create_model('ada')
print(ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,3916.0732,22023287.6602,4692.8976,0.7582,0.4531,0.5107
1,4360.9397,30278296.5118,5502.5718,0.7680,0.5005,0.5845
2,4888.5203,29214381.0110,5405.0329,0.7321,0.6025,0.7529
3,4234.5176,22927466.2701,4788.2634,0.7470,0.5134,0.5976
4,4145.7508,26324797.7268,5130.7697,0.6717,0.5014,0.5877
5,4573.0428,29825292.6109,5461.2538,0.7454,0.5455,0.6580
6,4415.2098,26515712.1111,5149.3409,0.7509,0.5335,0.6298
7,4627.4198,29244362.0079,5407.8057,0.6302,0.5479,0.6558
8,4010.2564,21764978.2303,4665.2951,0.7863,0.4908,0.5601
9,4292.5562,25271178.0599,5027.0447,0.7594,0.5362,0.6420


AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=50, random_state=123)


In [18]:
tuned_ada = tune_model(ada)
print(tuned_ada)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2628.2684,16209063.7948,4026.0482,0.8220,0.2553,0.2244
1,2740.5318,23655434.4913,4863.6853,0.8187,0.2715,0.2373
2,2605.9909,16883405.3119,4108.9421,0.8451,0.2617,0.2352
3,2576.8639,14368597.3247,3790.5933,0.8415,0.2688,0.2275
4,2392.3346,13555556.2417,3681.7871,0.8309,0.2670,0.2225
5,2538.7416,20724600.2592,4552.4280,0.8231,0.2644,0.2260
6,2720.2195,19796302.1522,4449.3036,0.8140,0.2644,0.2280
7,2707.6016,17084596.1502,4133.3517,0.7839,0.2743,0.2475
8,2444.0262,16340453.5625,4042.3327,0.8395,0.2623,0.2199
9,2547.2185,19272925.6516,4390.0940,0.8165,0.2683,0.2248


AdaBoostRegressor(base_estimator=None, learning_rate=0.05, loss='linear',
                  n_estimators=90, random_state=123)


In [19]:
predict_model(tuned_ada)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,AdaBoost Regressor,2742.6424,19003838.2784,4359.3392,0.8267,0.2711,0.2329


,Carat Weight,Cut_Fair,Cut_Good,Cut_Ideal,Cut_Signature-Ideal,Cut_Very Good,Color_D,Color_E,Color_F,Color_G,...,Polish_G,Polish_ID,Polish_VG,Symmetry_EX,Symmetry_G,Symmetry_ID,Symmetry_VG,Report_GIA,Price,Label
0,-0.354048,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,7059.0,7315.943631
1,-0.396450,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5809.0,7315.943631
2,-0.163237,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,10641.0,8641.463847
3,-1.138493,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,2871.0,4024.857724
4,1.914484,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,21706.0,27516.784226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616,-0.226840,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,12906.0,7350.228758
1617,-1.053688,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,3235.0,4024.857724
1618,0.387995,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,9058.0,12748.734375
1619,-0.354048,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5091.0,7315.943631


In [20]:

final_ada = finalize_model(tuned_ada)

In [21]:
unseen_predictions = predict_model(final_ada, data=data_unseen)
unseen_predictions.head()

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price,Label
0,1.53,Ideal,E,SI1,ID,ID,AGSL,12791,12888.906767
1,1.50,Fair,F,SI1,VG,VG,GIA,10450,12888.906767
2,1.01,Good,E,SI1,G,G,GIA,5161,7172.492974
3,2.51,Very Good,G,VS2,VG,VG,GIA,34361,30548.220465
4,1.01,Good,I,SI1,VG,VG,GIA,4238,7172.492974


In [22]:
from pycaret.utils import check_metric
check_metric(unseen_predictions.Price, unseen_predictions.Label, 'R2')

0.8285

In [24]:
save_model(final_ada,'Final ada Model 28Mar2021')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='Price',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='...
                 ('dummy', Dummify(target='Price')),
                 ('fix_perfect', Remove_100(target='Price')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough